# ⚛️ H₂O — Bond Angle Optimization via Noiseless VQE (UCCSD)

This notebook scans **H–O–H bond angles** and computes the **ground-state energy**
of the water molecule using the production VQE API:

```python
from vqe.core import run_vqe_geometry_scan
```

For each bond angle, the package:
- Builds the H₂O geometry (fixed O–H bond length, variable H–O–H angle)
- Constructs the molecular Hamiltonian in **STO-3G**
- Runs **noiseless VQE** with a **UCCSD** ansatz
- Averages over multiple seeds (if requested)
- Produces an **energy vs bond angle** plot with error bars

Goal: approximate the **equilibrium bond angle** of H₂O.

This notebook is a **pure client**: no custom QNodes, ansatzes, or caching logic.

In [ ]:
import numpy as np
import sys
import os

# Allow local import when running from notebooks/
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "../..")))

from vqe.core import run_vqe_geometry_scan

# 🧬 Angle Grid

We define a one-dimensional scan over the H–O–H bond angle.
These angles should be interpreted in **degrees**, consistent with
the geometry helper used inside the package.

In [ ]:
angles = np.linspace(100.0, 109.0, 5)  # degrees
print("H–O–H angle grid (deg):", angles)

# 🚀 Geometry Scan via `run_vqe_geometry_scan`

We now call the high-level geometry scan helper:

```python
run_vqe_geometry_scan(
    molecule="H2O_ANGLE",
    param_name="angle",
    param_values=angles,
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    steps=35,
    stepsize=0.2,
    seeds=[0],
)
```

This:
- Uses the internal **H₂O bond-angle geometry generator**
- Calls `run_vqe` under the hood for each angle / seed
- Reuses cached runs when available
- Produces and saves an **Energy vs bond angle** plot

In [ ]:
results = run_vqe_geometry_scan(
    molecule="H2O_ANGLE",
    param_name="angle",
    param_values=angles,
    ansatz_name="UCCSD",
    optimizer_name="Adam",
    steps=35,
    stepsize=0.2,
    seeds=[0],
    force=False,
    mapping="jordan_wigner",
    show=True,
)

# 📌 Extract Approximate Equilibrium Bond Angle

`run_vqe_geometry_scan` returns:

```python
results = [
    (angle, mean_energy, std_energy),
    ...
]
```

where:
- `angle` is the H–O–H bond angle in degrees
- `mean_energy` is the average ground-state energy over seeds
- `std_energy` is the standard deviation over seeds

In [ ]:
# Unpack results
params, mean_E, std_E = zip(*results)
params = np.array(params, dtype=float)
mean_E = np.array(mean_E, dtype=float)
std_E = np.array(std_E, dtype=float)

# Locate minimum mean energy
min_idx = int(np.argmin(mean_E))
opt_angle = params[min_idx]
opt_energy = mean_E[min_idx]
opt_sigma = std_E[min_idx]

print(f"\nEstimated equilibrium H–O–H bond angle: {opt_angle:.2f}°")
print(f"Minimum mean VQE ground-state energy: {opt_energy:.8f} ± {opt_sigma:.8f} Ha")

In [ ]:
import matplotlib.pyplot as plt

from common.molecule_viz import plot_molecule

# Fixed O–H bond length used by the geometry helper (Å)
r_OH = 0.958  # standard water bond length

# Convert angle to radians
theta = np.deg2rad(opt_angle)

# Place oxygen at origin, molecule in the x–y plane
symbols = ["O", "H", "H"]
coords = np.array([
    [0.0, 0.0, 0.0],
    [r_OH, 0.0, 0.0],
    [r_OH * np.cos(theta), r_OH * np.sin(theta), 0.0],
])

plot_molecule(
    symbols,
    coords,
    title=f"H₂O geometry at equilibrium angle ≈ {opt_angle:.2f}°",
    bonds=[(0, 1), (0, 2)],
    angles=[(1, 0, 2)],
    show_bond_lengths=True,
    show_atom_indices=False,
)

plt.show()


---
## Summary

In this notebook you:

- Used the **production VQE API** (`run_vqe_geometry_scan`) as a *pure client*
- Performed a **noiseless UCCSD VQE** bond-angle scan for **H₂O**
- Obtained a **Ground-state energy vs bond angle** curve (plot saved by the package)
- Extracted an approximate **equilibrium bond angle** with an energy estimate

All Hamiltonian construction, geometry handling, ansatz setup, device
management, optimisation, caching, and plotting were handled by the package.